**Update Feb 8: ** The sentiment analysis works but the original texts are all neutral so it won't have an impact of the predictions. However I will use it for now as a categorical value where "0" = Has captions and "-2" = "no caption".

Doing this analysis made me realize that I have to write captions where emotions are present so my audience feels more conected.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
from sklearn.naive_bayes import (GaussianNB, MultinomialNB)
import re 
import nltk 
from sklearn.metrics import (accuracy_score, ConfusionMatrixDisplay)
from nltk.corpus import stopwords
from nltk import ngrams
nltk.download("stopwords")
stop_words = stopwords.words("spanish") # stopwords palabras de relleno

# Análisis de texto clásico ciertas palabras no añaden contexto y en base al resto de palabras podemos entender las relaciones que permitan hacer la clasficación
# Función para quitar los stopwords del data frame
def remove_stopwords(text):
    clean_text = []
    text = text.lower()
    if text is not None:
        words = text.split()
        for word in words:
            if word not in stop_words:
                clean_text.append(word)
        return " ".join(clean_text)
    else:
        return None


def remove_symbols(text):
    if text is not None:
        text = re.sub(r'''[.,?*&'"+${}-]''','',text) # Sub sustituir 
        return text
    else:
        return None

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Open the Dataset
df_raw = pd.read_csv("archivo_combinado.csv")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Identificador de la publicación  702 non-null    int64  
 1   Identificador de la cuenta       702 non-null    int64  
 2   Nombre de usuario de la cuenta   702 non-null    object 
 3   Nombre de la cuenta              702 non-null    object 
 4   Descripción                      302 non-null    object 
 5   Duración (segundos)              702 non-null    int64  
 6   Hora de publicación              702 non-null    object 
 7   Enlace permanente                13 non-null     object 
 8   Tipo de publicación              702 non-null    object 
 9   Comentario sobre los datos       0 non-null      float64
 10  Fecha                            702 non-null    object 
 11  Alcance                          642 non-null    float64
 12  Me gusta              

In [4]:
# Create a Dataframe only with the id and description
selected_columns = ["Identificador de la publicación", "Descripción"]
df_interim = df_raw[selected_columns]
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Identificador de la publicación  702 non-null    int64 
 1   Descripción                      302 non-null    object
dtypes: int64(1), object(1)
memory usage: 11.1+ KB


In [5]:
# Remove rows with NaN
df_interim = df_interim.dropna(subset=["Descripción"])
df_interim.info()


<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 701
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Identificador de la publicación  302 non-null    int64 
 1   Descripción                      302 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.1+ KB


In [6]:
# Clean data
df_interim["message_clean"] = df_interim["Descripción"].apply(lambda x: remove_stopwords(x))
df_interim["message_clean"] = df_interim["message_clean"].apply(lambda x: remove_symbols(x))
df_interim["message_clean"] = df_interim["message_clean"].str.replace(":", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("!", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("¡", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace("(", "")
df_interim["message_clean"] = df_interim["message_clean"].str.replace(")", "")
df_interim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 302 entries, 0 to 701
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Identificador de la publicación  302 non-null    int64 
 1   Descripción                      302 non-null    object
 2   message_clean                    302 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.4+ KB


In [7]:
df_interim.sample(10)

,Identificador de la publicación,Descripción,message_clean
417,18024890534096696,"Si quieres compartir tu respuesta, ve a mi can...",si quieres compartir respuesta ve canal ig
373,17962323986187219,Oficialmente se acabó el Guadalupe -Reyes,oficialmente acabó guadalupe reyes
518,17957312969373800,Me siento muy contenta de formar parte del equ...,siento contenta formar parte equipo @ekrmexico
579,18054681259789966,Hoy agradezco todo el apoyo que me brindan en ...,hoy agradezco apoyo brindan maternidad @titere...
583,18050136700849085,"Hoy agradezco ser mamá, aún con todo lo difíci...",hoy agradezco ser mamá aún difícil resulta vec...
260,17991133759882374,"Hoy es Viernes de dolores, en mi ciudad regala...",hoy viernes dolores ciudad regalan helado
38,17984448551572170,¡Tengo algo\nPara ti en\nla siguiente\nHistoria!,tengo siguiente historia
442,18029196935465461,¡Buen\nFinde!,buen finde
426,17989265882537049,¡Sábado social!,sábado social
101,18307794460112164,Bonita tarde,bonita tarde


In [9]:
import spacy

# Carga el modelo preentrenado en español
nlp = spacy.load("es_core_news_sm")

# Función para obtener el sentimiento de un texto
def obtener_sentimiento(texto):
    if isinstance(texto, str) and texto.strip():
        doc = nlp(texto)
        return doc.sentiment
    else:
        return -2

# Cargar el DataFrame original desde el archivo CSV
df_original = pd.read_csv('archivo_combinado.csv')

# Agregar la columna 'sentimiento' al df_interim
df_interim['sentimiento'] = df_interim['message_clean'].apply(obtener_sentimiento)

# Combinar el DataFrame original con el df_interim usando el Identificador de la publicación
df_actualizado = df_original.merge(df_interim[['Identificador de la publicación', 'sentimiento']], on='Identificador de la publicación', how='left')

# Asignar -2 a las publicaciones sin captions (donde el sentimiento es NaN)
df_actualizado['sentimiento'].fillna(-2, inplace=True)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
df_actualizado.to_csv('archivo_combinado_updated.csv', index=False)

# Mostrar el DataFrame con la nueva columna de sentimiento
print(df_actualizado.head())


   Identificador de la publicación  Identificador de la cuenta  \
0                18003913832026677           17841408055500202   
1                17896594586939091           17841408055500202   
2                17943046658757431           17841408055500202   
3                17886212690976454           17841408055500202   
4                18112602532344424           17841408055500202   

  Nombre de usuario de la cuenta  \
0                    jessonirica   
1                    jessonirica   
2                    jessonirica   
3                    jessonirica   
4                    jessonirica   

                                 Nombre de la cuenta  \
0  Jess Onírica | Sueños, creatividad, espiritual...   
1  Jess Onírica | Sueños, creatividad, espiritual...   
2  Jess Onírica | Sueños, creatividad, espiritual...   
3  Jess Onírica | Sueños, creatividad, espiritual...   
4  Jess Onírica | Sueños, creatividad, espiritual...   

                                         Descripc

/tmp/ipykernel_3694/2039047640.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_actualizado['sentimiento'].fillna(-2, inplace=True)
